In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('churn_analysis.csv', index_col=0)

In [3]:
df.shape


(3333, 22)

In [4]:
control = df[df['treatment']==1]
states = control.state.unique()

In [5]:
sdf = pd.DataFrame(columns=['state', 'churn', 'no_churn'])
for s in states:
    sdf = sdf.append({'state': s, 
                'churn': control[(control['state']==s)&(control['churn']=='True.')].shape[0], 
                'no_churn': control[(control['state']==s)&(control['churn']=='False.')].shape[0]}, 
              ignore_index=True);
    
    

/var/folders/2d/8s7fzr_974372gjw0dqzc7g00000gn/T/ipykernel_2049/3351616026.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdf = sdf.append({'state': s,
/var/folders/2d/8s7fzr_974372gjw0dqzc7g00000gn/T/ipykernel_2049/3351616026.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdf = sdf.append({'state': s,
/var/folders/2d/8s7fzr_974372gjw0dqzc7g00000gn/T/ipykernel_2049/3351616026.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdf = sdf.append({'state': s,
/var/folders/2d/8s7fzr_974372gjw0dqzc7g00000gn/T/ipykernel_2049/3351616026.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sdf = sdf.append({'state': s,
/var/folders/2d/

In [6]:
sdf.head()

,state,churn,no_churn
0,KS,7,18
1,OH,2,22
2,LA,0,15
3,IA,2,13
4,VT,5,18


In [70]:
from itertools import combinations
import numpy as np
from scipy.stats import chi2_contingency, fisher_exact
comb = combinations(states, 2)
p_less = 0
p_less_corrected = 0
plist=[]
plist_corrected=[]
for i in comb:
    l = []
    first_s = sdf[sdf['state']==i[0]][['churn', 'no_churn']]
    second_s = sdf[sdf['state']==i[1]][['churn', 'no_churn']]
    l.append(first_s.values.tolist()[0])
    l.append(second_s.values.tolist()[0])
    chi2, p, dof, ex = chi2_contingency(l, correction=False)
    plist.append(p)
    p_less += int(p < 0.05)
    chi2_corrected, p_corrected, dof_corrected, ex_corrected = chi2_contingency(l, correction=True)
    plist_corrected.append(p_corrected)
    p_less_corrected += int(p_corrected < 0.05)

In [72]:
print(np.mean(plist), np.mean(plist_corrected))

0.5018273798739158 0.7207040857368472


In [11]:
print(p_less, p_less_corrected)

34 0


In [23]:
comb = combinations(states, 2)

p_l = 0
for i in comb:

    l = []
    first_s = sdf[sdf['state']==i[0]][['churn', 'no_churn']]
    second_s = sdf[sdf['state']==i[1]][['churn', 'no_churn']]
    l.append(first_s.values.tolist()[0])
    l.append(second_s.values.tolist()[0])
    o, p = fisher_exact(l)
    p_l += int(p < 0.05)

In [24]:
p_l

10

In [41]:
from scipy.stats import pearsonr, spearmanr, mannwhitneyu
corr, p = pearsonr(df['day_calls'], df['mes_estim'])
print(corr, p)

-0.051794350587572625 0.0027798836869738384


In [74]:
corr, p = spearmanr( df['mes_estim'],df['day_calls'])
print(corr, p)

0.043349880533927444 0.012317367189170541


In [39]:
import math
m = np.array(pd.crosstab(df[df['treatment']==1]['state'], df[df['treatment']==1]['churn']))
n = m.sum()
chi2, p, dof, ex = chi2_contingency(m)
cramer = math.sqrt(chi2/(n*(min(7,2) - 1)))
print(f'Chi_2 статистика: {chi2} \nДостигаемый уровень значимости: {p} \nКоэффициент V Крамера: {cramer}')

Chi_2 статистика: 44.05271236692548 
Достигаемый уровень значимости: 0.7097590042778473 
Коэффициент V Крамера: 0.2003932150203332


In [66]:
t1 = [0 if i=='False.' else 1 for i in df[df['treatment']==1]['churn']]
t0 = [0 if i=='False.' else 1 for i in df[df['treatment']==0]['churn']]
t2 = [0 if i=='False.' else 1 for i in df[df['treatment']==2]['churn']]


import scipy


In [67]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))


def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)


In [69]:
print("p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_ind(t0, t1)))
print("p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_ind(t1, t2)))
print("p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_ind(t0, t2)))

p-value: 0.228331
p-value: 0.009348
p-value: 0.156425
